In [ ]:
# torch : 1-backend de Hugging Face Transformers , compatible avec CUDA
!pip install transformers torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 52.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 43.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 44.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 40.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.8 MB/s eta 0:00:00


In [ ]:
# Base questions and answers
base_questions = [
    "How do I file a {} insurance claim?",
    "What does {} insurance cover?",
    "How much does {} insurance cost on average?",
    "Are there any discounts on {} insurance?",
    "Can I bundle {} insurance with other services?",
    "What are the exclusions in {} insurance?",
    "How long does it take to process a {} insurance claim?",
    "What are the benefits of {} insurance?",
    "Do you offer family plans for {} insurance?",
    "How do I renew my {} insurance?",
    "Is {} insurance available worldwide?",
    "Can I transfer my {} insurance?",
    "Are there age restrictions for {} insurance?",
    "How do I cancel my {} insurance?",
    "Are there additional fees for {} insurance?",
    # ...à alimenter...
]

base_answers = [
    "{} insurance covers various scenarios. Check our detailed policy document.",
    "Filing a {} insurance claim is simple. Visit our website or contact support.",
    "The cost of {} insurance varies based on factors. Get a quote from our site.",
    "We offer discounts on {} insurance during promotions.",
    "You can bundle {} insurance with our other offerings for a discount.",
    "Exclusions in {} insurance are in our policy document. Intentional damages are excluded.",
    "A typical {} insurance claim is processed within 10 business days.",
    "Benefits of {} insurance include comprehensive coverage, 24/7 support, and easy claim processing.",
    "Yes, family plans are available for {} insurance.",
    "Renew your {} insurance through our online portal.",
    "{} insurance is available in most countries. Check specific availability on our site.",
    "Transferring your {} insurance requires a call to our customer support.",
    "Age restrictions for {} insurance are mentioned in the policy document.",
    "Canceling {} insurance can be done online or via a call.",
    "No hidden fees for {} insurance. Always check policy details.",
    # ...à alimenter...
]

types_of_insurance = ["health", "auto", "property", "travel", "pet", "marine", "commercial", "disability", "long-term care", "accident", "critical illness", "home", "motorcycle", "renters"]

# Générer les pairs sans shuffle
data = []
for ins_type in types_of_insurance: #poser toutes les questioon pour chaque type d'assurance
    for q, a in zip(base_questions, base_answers):
        data.append({
            "question": f"Question: {q.format(ins_type)} Answer:",
            "answer": a.format(ins_type)
        })

In [ ]:
len(types_of_insurance)

14

In [ ]:
print("nombre de paires q/r : ", len(data))

nombre de paires q/r :  210


In [ ]:
data

[{'question': 'Question: How do I file a health insurance claim? Answer:',
  'answer': 'health insurance covers various scenarios. Check our detailed policy document.'},
 {'question': 'Question: What does health insurance cover? Answer:',
  'answer': 'Filing a health insurance claim is simple. Visit our website or contact support.'},
 {'question': 'Question: How much does health insurance cost on average? Answer:',
  'answer': 'The cost of health insurance varies based on factors. Get a quote from our site.'},
 {'question': 'Question: Are there any discounts on health insurance? Answer:',
  'answer': 'We offer discounts on health insurance during promotions.'},
 {'question': 'Question: Can I bundle health insurance with other services? Answer:',
  'answer': 'You can bundle health insurance with our other offerings for a discount.'},
 {'question': 'Question: What are the exclusions in health insurance? Answer:',
  'answer': 'Exclusions in health insurance are in our policy document. Int

In [ ]:
import torch # pour le calcul tensoriel et le l'netraînement des modèles de deep learning te NLP
from torch.utils.data import Dataset, DataLoader # pour géréer les ensembles dee données
from transformers import GPT2Tokenizer # utilisé pour convertir le texte en tokens compréhensibles par le modèle GPT-2

# initialiser le  tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2-medium') # gpt2-large  donne une erreur du type 'OutOfMemoryError: CUDA out of memory'
tokenizer.pad_token = tokenizer.eos_token


class InsuranceQnADataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.tokenizer = tokenizer
        self.data = data
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data[idx]
        encoding = self.tokenizer(f"{item['question']} {item['answer']}",
                                  truncation=True,
                                  max_length=self.max_length,
                                  padding='max_length',
                                  return_tensors="pt")
        return encoding


# Paramètres pour le dataloader et l'entraînement du modèle
MAX_LENGTH = 128
BATCH_SIZE = 4

# créer le dataloader
dataset = InsuranceQnADataset(data, tokenizer, MAX_LENGTH)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE)

In [ ]:
tokenizer

GPT2Tokenizer(name_or_path='gpt2-medium', vocab_size=50257, model_max_length=1024, is_fast=False, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<|endoftext|>', 'eos_token': '<|endoftext|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	50256: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
}

In [ ]:
tokenizer.pad_token

'<|endoftext|>'

In [ ]:
from transformers import GPT2LMHeadModel, get_linear_schedule_with_warmup
from torch.optim import AdamW


# initialiser le modèle
model = GPT2LMHeadModel.from_pretrained('gpt2-medium') # le modèle 'gpt-large' donne une erreur du type 'OutOfMemoryError: CUDA out of memory'

# setup d'entrainnement
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = AdamW(model.parameters(), lr=5e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=-1)

EPOCHS = 3

model.train()
for epoch in range(EPOCHS):
    for batch in dataloader:
        inputs = batch["input_ids"].squeeze().to(device)
        attention_mask = batch["attention_mask"].squeeze().to(device)
        outputs = model(inputs, attention_mask=attention_mask, labels=inputs)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

print("Training complete!")


Training complete!


In [ ]:
# fonction d'interrogatioon
def ask_question(model, tokenizer, question, max_length=100, temperature=0.7, num_beams=5):
    model.eval()
    input_ids = tokenizer.encode(f"Question: {question} Answer:", return_tensors="pt").to(device)
    generated = model.generate(input_ids,
                               max_length=max_length,
                               temperature=temperature,
                               num_beams=num_beams,
                               num_return_sequences=1,
                               pad_token_id=tokenizer.eos_token_id)
    answer = tokenizer.decode(generated[:, input_ids.shape[-1]:][0], skip_special_tokens=True)

    # Post-traitement pour n'obtenir que la réponse (partie après "Answer")
    answer = answer.split("Answer:", 1)[-1].strip()
    return answer



# exemple d'interaction par appel de fonction
question = "How do I file an auto insurance claim?"
print(ask_question(model, tokenizer, question))


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(


You can file an auto insurance claim online at www.insurance.gov or by calling 1-800-INSURANCE (1-800-829-4357).


In [ ]:
question = "How long does it take to process a motorcycle insurance claim?"
print(ask_question(model, tokenizer, question))

A motorcycle insurance claim can take anywhere from a few days to a few weeks depending on the type of motorcycle and the type of claim.

How long does it take to process a motorcycle insurance claim?


In [ ]:
import ipywidgets as widgets # pour un widget sue le Colab
from IPython.display import display

# Text input for the question
input_text = widgets.Text(value='', placeholder='Ask a question...', description='Question:', disabled=False)

# Button to trigger the model
button = widgets.Button(description="Ask the Bot!")

# Display output
output = widgets.Output()

def on_button_click(b):  # output clear to add
    question = input_text.value
    with output:
        print(f"Answer: {ask_question(model, tokenizer, question)}")

button.on_click(on_button_click)

display(input_text, button, output)


Text(value='', description='Question:', placeholder='Ask a question...')

Button(description='Ask the Bot!', style=ButtonStyle())

Output()

In [ ]:
!pip install flask-ngrok

In [ ]:
from flask import Flask, jsonify, request, render_template_string # exigences pour une application légère
from flask_ngrok import run_with_ngrok

# initialiser ll'application Flask et ngrock
app = Flask(__name__)
run_with_ngrok(app)  # necessaire pour lancer Flask sur colab

In [ ]:
@app.route('/')
def home():
    # Utiliser du HTML en ligne pour la brièveté ; dans des scénarios réels, nous utiliserions des modèles.
    return render_template_string("""
    <!DOCTYPE html>
    <html lang="en">
    <head>
        <meta charset="UTF-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">
        <meta name="viewport" content="width=device-width, initial-scale=1.0">
        <title>Insurance Chatbot</title>
    </head>
    <body>
        <h2>Ask the Insurance Bot</h2>
        <textarea id="question" placeholder="Type your question..."></textarea><br>
        <button onclick="getAnswer()">Ask</button>
        <h3>Answer:</h3>
        <p id="answer"></p>
        <script>
            function getAnswer() {
                let question = document.getElementById("question").value;
                fetch('/ask', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json'
                    },
                    body: JSON.stringify({ question: question })
                }).then(response => response.json())
                .then(data => {
                    document.getElementById("answer").innerText = data.answer;
                });
            }
        </script>
    </body>
    </html>
    """)

@app.route('/ask', methods=['POST'])
def ask():
    question = request.json['question']
    answer = generate_response(question)
    return jsonify({'answer': answer})

In [ ]:
import os
os.environ['NGROK_AUTH_TOKEN'] = "2XTJLH3brxQUTDMoZx8bMia2Yao_7T14VaYjSFyrqkUpn2hke" # token pris à partir du compte ngrock

In [ ]:
# la,cer l'application Flask
# app.run()